In [ ]:
pip install pillow numpy matplotlib

In [ ]:
import os
from PIL import Image
from collections import defaultdict
import gc
import re

count = 1  # For counting processed files

# 🔍 Extract patient, trial, condition, channel from filename
def parse_filename(filename):
    match = re.match(r"sub(\d+)_trial(\d+)_cond(\d+\.\d+)_ch(\d+)\.png", filename)
    if match:
        patient = f"sub{match.group(1)}"
        trial = int(match.group(2))
        condition = float(match.group(3))
        channel = int(match.group(4))
        return patient, trial, condition, channel
    return None

# 🧩 Combine 1x19 colored images
def combine_1x19_and_resize(image_paths, image_size=(224, 224), output_size=(224, 224)):
    block_w, block_h = image_size
    canvas = Image.new('RGB', (block_w * 19, block_h), (0, 0, 0))
    black = Image.new('RGB', image_size, (0, 0, 0))

    for idx in range(19):
        if image_paths[idx] is not None:
            img = Image.open(image_paths[idx]).resize(image_size)
        else:
            img = black.copy()
        x = idx * block_w
        canvas.paste(img, (x, 0))
        img.close()

    resized = canvas.resize(output_size)
    return resized

# 🚀 Main processing function
def generate_combined_images(image_dir, output_dir, image_size=(224, 224), output_size=(224, 224)):
    os.makedirs(output_dir, exist_ok=True)
    file_map = defaultdict(lambda: [None]*19)
    global count

    for fname in os.listdir(image_dir):
        if fname.endswith('.png'):
            parsed = parse_filename(fname)
            if parsed:
                patient, trial, condition, channel = parsed
                if 0 <= channel <= 18:
                    key = (patient, trial, condition)
                    file_map[key][channel] = os.path.join(image_dir, fname)

    print(f"🧠 Found {len(file_map)} combinations to process.")

    for (patient, trial, condition), paths in file_map.items():
        out_fname = f"{patient}_trial{trial}_cond{condition}_1x19.png"
        out_path = os.path.join(output_dir, out_fname)
        if os.path.exists(out_path):
            continue  # Skip if already processed

        try:
            combined_img = combine_1x19_and_resize(paths, image_size=image_size, output_size=output_size)
            combined_img.save(out_path)
            combined_img.close()
            del combined_img
            gc.collect()

            print(f"✅ {count}. Saved {out_fname}")
            count += 1

        except Exception as e:
            print(f"❌ Error processing {patient}_trial{trial}_cond{condition}: {e}")
            continue

    print("🎉 Done combining all image sets.")

# 🔧 Example usage
if __name__ == "__main__":
    image_dir = "/home/zeek/External/Top20/schizophrenic"
    output_dir = "/home/zeek/External/Top20_combined/schizophrenic"
    generate_combined_images(image_dir, output_dir)


🧠 Found 2817 combinations to process.
✅ 1. Saved sub25_trial10_cond1.0_1x19.png
✅ 2. Saved sub25_trial10_cond2.0_1x19.png
✅ 3. Saved sub25_trial10_cond3.0_1x19.png
✅ 4. Saved sub25_trial11_cond1.0_1x19.png
✅ 5. Saved sub25_trial11_cond2.0_1x19.png
✅ 6. Saved sub25_trial11_cond3.0_1x19.png
✅ 7. Saved sub25_trial12_cond1.0_1x19.png
✅ 8. Saved sub25_trial12_cond2.0_1x19.png
✅ 9. Saved sub25_trial12_cond3.0_1x19.png
✅ 10. Saved sub25_trial13_cond1.0_1x19.png
✅ 11. Saved sub25_trial13_cond2.0_1x19.png
✅ 12. Saved sub25_trial13_cond3.0_1x19.png
✅ 13. Saved sub75_trial9_cond1.0_1x19.png
✅ 14. Saved sub75_trial9_cond2.0_1x19.png
✅ 15. Saved sub76_trial1_cond2.0_1x19.png
✅ 16. Saved sub76_trial10_cond2.0_1x19.png
✅ 17. Saved sub76_trial10_cond3.0_1x19.png
✅ 18. Saved sub76_trial11_cond1.0_1x19.png
✅ 19. Saved sub76_trial11_cond2.0_1x19.png
✅ 20. Saved sub76_trial11_cond3.0_1x19.png
✅ 21. Saved sub76_trial12_cond1.0_1x19.png
✅ 22. Saved sub76_trial12_cond2.0_1x19.png
✅ 23. Saved sub76_trial12_co

In [ ]:
!pip install opencv-python